### 데이터 가져오기

In [1]:
import gdown

gdown.download('https://bit.ly/3q9SZix', '20s_best_book.json', quiet=False)

Downloading...
From: https://bit.ly/3q9SZix
To: /Volumes/Data/22.HYMOT/03.Python/01.Coursework/202309.Python/Exercise13/20s_best_book.json
100%|██████████████████████████████████████| 92.9k/92.9k [00:00<00:00, 9.03MB/s]


'20s_best_book.json'

In [2]:
import pandas as pd

books_df = pd.read_json('20s_best_book.json')
books_df.head()

,no,ranking,bookname,authors,publisher,publication_year,isbn13,addition_symbol,vol,class_no,class_nm,loan_count,bookImageURL
0,1,1,우리가 빛의 속도로 갈 수 없다면 :김초엽 소설,지은이: 김초엽,허블,2019,9791190090018,03810,,813.7,문학 > 한국문학 > 소설,461,https://image.aladin.co.kr/product/19359/16/co...
1,2,2,달러구트 꿈 백화점.이미예 장편소설,지은이: 이미예,팩토리나인,2020,9791165341909,03810,,813.7,문학 > 한국문학 > 소설,387,https://image.aladin.co.kr/product/24512/70/co...
2,3,3,지구에서 한아뿐 :정세랑 장편소설,지은이: 정세랑,난다,2019,9791188862290,03810,,813.7,문학 > 한국문학 > 소설,383,https://image.aladin.co.kr/product/19804/82/co...
3,4,4,"시선으로부터, :정세랑 장편소설",지은이: 정세랑,문학동네,2020,9788954672214,03810,,813.7,문학 > 한국문학 > 소설,370,https://image.aladin.co.kr/product/24131/37/co...
4,5,5,아몬드 :손원평 장편소설,지은이: 손원평,창비,2017,9788936434267,03810,,813.7,문학 > 한국문학 > 소설,365,http://image.aladin.co.kr/product/16839/4/cove...


In [3]:
books = books_df[['no','ranking','bookname','authors','publisher',
                 'publication_year','isbn13']]
books.head()

,no,ranking,bookname,authors,publisher,publication_year,isbn13
0,1,1,우리가 빛의 속도로 갈 수 없다면 :김초엽 소설,지은이: 김초엽,허블,2019,9791190090018
1,2,2,달러구트 꿈 백화점.이미예 장편소설,지은이: 이미예,팩토리나인,2020,9791165341909
2,3,3,지구에서 한아뿐 :정세랑 장편소설,지은이: 정세랑,난다,2019,9791188862290
3,4,4,"시선으로부터, :정세랑 장편소설",지은이: 정세랑,문학동네,2020,9788954672214
4,5,5,아몬드 :손원평 장편소설,지은이: 손원평,창비,2017,9788936434267


### 도서 검색

In [4]:
import requests

isbn = 9791190090018      # '우리가 빛의 속도로 갈 수 없다면'의 ISBN
url = 'http://www.yes24.com/Product/Search?domain=BOOK&query={}'

r = requests.get(url.format(isbn))

### 상세 검색

In [5]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(r.text, 'html.parser')
prd_link = soup.find('a', attrs={'class':'gd_name'})

In [13]:
prd_link

<a class="gd_name" href="/Product/Goods/74261416" onclick="wiseLogV2('S', '101_005_003_001', ''); setGoodsClickExtraCodeHub('032', '9791190090018', '74261416', '0');">우리가 빛의 속도로 갈 수 없다면</a>

In [6]:
# '우리가 빛의 속도로 갈 수 없다면'의 상세 페이지 가져오기
url = 'http://www.yes24.com'+prd_link['href']
r = requests.get(url)

In [7]:
soup = BeautifulSoup(r.text, 'html.parser')
prd_detail = soup.find('div', attrs={'id':'infoset_specific'})

In [8]:
prd_tr_list = prd_detail.find_all('tr')

In [9]:
for tr in prd_tr_list:
    if tr.find('th').get_text() == '쪽수, 무게, 크기':
        page_td = tr.find('td').get_text()
        break

In [10]:
print(page_td)

330쪽 | 496g | 130*198*30mm


In [14]:
# 무게 가져오기
print(page_td.split()[2])

496g


### (1) 다른 정보 추출하기

#### 무게 추출하기

In [15]:
def get_weight(isbn):
    # Yes24 도서 검색 페이지 URL
    url = 'http://www.yes24.com/Product/Search?domain=BOOK&query={}'
    # URL에 ISBN을 넣어 HTML 가져옵니다.
    r = requests.get(url.format(isbn))
    soup = BeautifulSoup(r.text, 'html.parser')   # HTML 파싱
    # 검색 결과에서 해당 도서를 선택합니다.
    prd_info = soup.find('a', attrs={'class':'gd_name'})
    if prd_info == None:
        return ''
    # 도서 상세 페이지를 가져옵니다.
    url = 'http://www.yes24.com'+prd_info['href']
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    # 상품 상세정보 div를 선택합니다.
    prd_detail = soup.find('div', attrs={'id':'infoset_specific'})
    # 테이블에 있는 tr 태그를 가져옵니다.
    prd_tr_list = prd_detail.find_all('tr')
    # 무게가 들어 있는 th를 찾아 td에 담긴 값을 반환합니다.
    for tr in prd_tr_list:
        if tr.find('th').get_text() == '쪽수, 무게, 크기':
            return tr.find('td').get_text().split()[2]
    return ''

In [20]:
# 하나의 책에 대해 무게 가져오기
get_weight(9791190090018)

'496g'

In [21]:
# 10개만
top10_books = books.head(10)

In [23]:
# 10개에 대한 무게 가져오고, 이름 지정
weight = top10_books.apply(lambda row: get_weight(row['isbn13']), axis=1)
weight.name = 'weight'

In [24]:
# 기존 데이터랑 합치기
top10_with_weight = pd.merge(top10_books, weight, left_index=True, right_index=True)
top10_with_weight

,no,ranking,bookname,authors,publisher,publication_year,isbn13,weight
0,1,1,우리가 빛의 속도로 갈 수 없다면 :김초엽 소설,지은이: 김초엽,허블,2019,9791190090018,496g
1,2,2,달러구트 꿈 백화점.이미예 장편소설,지은이: 이미예,팩토리나인,2020,9791165341909,358g
2,3,3,지구에서 한아뿐 :정세랑 장편소설,지은이: 정세랑,난다,2019,9791188862290,296g
3,4,4,"시선으로부터, :정세랑 장편소설",지은이: 정세랑,문학동네,2020,9788954672214,412g
4,5,5,아몬드 :손원평 장편소설,지은이: 손원평,창비,2017,9788936434267,388g
5,6,6,피프티 피플 :정세랑 장편소설,지은이: 정세랑,창비,2016,9788936434243,512g
6,7,7,목소리를 드릴게요 :정세랑 소설집,지은이: 정세랑,아작,2020,9791165300005,318g
7,8,8,나미야 잡화점의 기적 :히가시노 게이고 장편소설,지은이: 히가시노 게이고 ;옮긴이: 양윤옥,현대문학,2012,9788972756194,
8,9,9,선량한 차별주의자,김지혜 지음,창비,2019,9788936477196,324g
9,10,9,쇼코의 미소 :최은영 소설,지은이: 최은영,문학동네,2016,9788954641630,406g


### (2) 쪽수와 다른 컬럼 같이 추가하기

#### 무게와 쪽수 추출해서 리스트로 저장

In [25]:
def get_weight_page_cnt(isbn):
    # Yes24 도서 검색 페이지 URL
    url = 'http://www.yes24.com/Product/Search?domain=BOOK&query={}'
    # URL에 ISBN을 넣어 HTML 가져옵니다.
    r = requests.get(url.format(isbn))
    soup = BeautifulSoup(r.text, 'html.parser')   # HTML 파싱
    # 검색 결과에서 해당 도서를 선택합니다.
    prd_info = soup.find('a', attrs={'class':'gd_name'})
    if prd_info == None:
        return ''
    # 도서 상세 페이지를 가져옵니다.
    url = 'http://www.yes24.com'+prd_info['href']
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    # 상품 상세정보 div를 선택합니다.
    prd_detail = soup.find('div', attrs={'id':'infoset_specific'})
    # 테이블에 있는 tr 태그를 가져옵니다.
    prd_tr_list = prd_detail.find_all('tr')
    # 쪽수가 들어 있는 th를 찾아 td에 담긴 값을 반환합니다.
    for tr in prd_tr_list:
        if tr.find('th').get_text() == '쪽수, 무게, 크기':
            return [tr.find('td').get_text().split()[2],tr.find('td').get_text().split()[0]]
    return ''

In [33]:
# 하나의 책에 대해 무게 가져오기
get_weight_page_cnt(9791190090018)

['496g', '330쪽']

In [ ]:
# 10개에 대한 무게와 쪽수 가져오기
weight_page_cnt = top10_books.apply(lambda row: get_weight_page_cnt(row['isbn13']), axis=1)

In [ ]:
# 데이터프레임으로 변경 및 이름 지정
w_df = weight_page_cnt.apply(lambda x:pd.Series(x))
w_df.columns = ['weight', 'page_cnt']

In [ ]:
w_df

In [ ]:
# NaN 공백으로 치환
import numpy as np

top10_with_weight_page_cnt = pd.merge(top10_books, w_df, left_index=True, right_index=True).replace(np.nan, '')
top10_with_weight_page_cnt